In [ ]:
# Source-to-parcel analysis

# Import necessary libraries
from matplotlib.animation import FuncAnimation
# import seaborn as sns  # required for heatmap visualization
import networkx as nx
from scipy.stats import pearsonr
from mne.viz import circular_layout
import pandas as pd
from mne_connectivity.viz import plot_connectivity_circle
import matplotlib.pyplot as plt
import os
import glob
import numpy as npc
import cupy as np #using gpu acceleration
import cupyx.scipy.fft
import mne
from mne.datasets import fetch_fsaverage
from nilearn import datasets
from nilearn.image import get_data
from scipy.signal import hilbert #scipy core modified in env, running custom lib
import scipy
import matplotlib
import os.path as op

matplotlib.use('Agg')  # Setting the backend BEFORE importing pyplot



scipy.fft.set_backend(cupyx.scipy.fft)

fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")


# Import necessary Python modules
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')


# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


def compute_cross_correlation(data_window):
    """Compute cross-correlation for given data window."""
    # Reshape the data to be 2D
    
    
    data_2D = data_window.reshape(data_window.shape[0], -1)
    correlation_matrix = np.corrcoef(data_2D, rowvar=True)
    return correlation_matrix

        # Compute dPLI at the level of regions


def compute_dPLI(data):
    print('Computing dPLI')
    n_regions = data.shape[1]  # Compute for regions
    dPLI_matrix = np.zeros((n_regions, n_regions))
    print(data)
    analytic_signal = hilbert(data)
    phase_data = np.angle(analytic_signal)
    for i in range(n_regions):
        for j in range(n_regions):
            if i != j:
                phase_diff = phase_data[:, i] - phase_data[:, j]
                dPLI_matrix[i, j] = np.abs(
                    np.mean(np.exp(complex(0, 1) * phase_diff)))
    return dPLI_matrix

# dPLI_matrix = compute_dPLI(label_time_courses) --> computing static, fc for the entire dataset


def disparity_filter(G, alpha=0.01):
    disparities = {}
    for i, j, data in G.edges(data=True):
        weight_sum_square = sum(
            [d['weight']**2 for _, _, d in G.edges(i, data=True)])
        disparities[(i, j)] = data['weight']**2 / weight_sum_square

    G_filtered = G.copy()
    for (i, j), disparity in disparities.items():
        if disparity < alpha:
            G_filtered.remove_edge(i, j)
    return G_filtered


def graph_to_matrix(graph, size):
    matrix = np.zeros((size, size))
    for i, j, data in graph.edges(data=True):
        matrix[i, j] = data['weight']
        matrix[j, i] = data['weight']  # Ensure symmetry
    return matrix

def plot_matrix(matrix, title, labels, cmap='viridis'):
    plt.figure(figsize=(10, 10))
    #plt.imsave(matrix, cmap='autumn')
    sns.heatmap(matrix, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.savefig(output_path+title+'.png')

def update(window_number):
    ax.clear()
    current_matrix = graph_to_matrix(nx.convert_matrix.from_numpy_array(
        windowed_cross_correlation_matrices[window_number]), windowed_cross_correlation_matrices[window_number].shape[0])
    sns.heatmap(current_matrix, cmap='viridis',
                xticklabels=ordered_regions, yticklabels=ordered_regions, ax=ax)
    title.set_text(
        f'Thresholded Cross-Correlation Matrix for Window {window_number}')
    return ax

def update(window_number):
    ax.clear()
    current_matrix = threshold_matrix(windowed_dpli_matrices[window_number])
    plot_connectivity_circle(current_matrix, ordered_regions, n_lines=300, node_angles=node_angles,
                            title=f'Thresholded Regional Connectivity using dPLI for Window {window_number}', ax=ax)
    return ax,


def threshold_matrix(matrix):
    G_temp = nx.convert_matrix.from_numpy_array(matrix)
    G_temp_thresholded = disparity_filter(G_temp)

    matrix_thresholded = np.zeros_like(matrix)
    for i, j, data in G_temp_thresholded.edges(data=True):
        matrix_thresholded[i, j] = data['weight']
        matrix_thresholded[j, i] = data['weight']
    return matrix_thresholded

def threshold_graph_by_density(G, density=0.1, directed=False):
    if density < 0 or density > 1:
        raise ValueError("Density value must be between 0 and 1.")
    num_edges_desired = int(G.number_of_edges() * density)
    sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'], 
reverse=True)
    if directed:
        G_thresholded = nx.DiGraph()
    else:
        G_thresholded = nx.Graph()
    G_thresholded.add_edges_from(sorted_edges[:num_edges_desired])
    return G_thresholded


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}


schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100)
fs_dir = '../data/in/fsaverage'
fname = os.path.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(fname, patch_stats=False, verbose=None)

for subject in subject_list:
    for mode in modes:
        print(subject, mode)
        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        stc_path  = output_path +'stc/'

        inverse_solution_files_lh = []
        inverse_solution_files_rh = []
        
        
        for path, subdirs, files in os.walk(stc_path):
            for file in files:
                filepath = path + file
                if '-rh.stc' in file:
                    inverse_solution_files_rh.append(filepath)
                elif '-lh.stc' in file:
                    inverse_solution_files_lh.append(filepath)

        # Error here !!!

        # Calculate the total number of inverse solution files for both hemispheres
        total_files_lh = len(inverse_solution_files_lh)
        total_files_rh = len(inverse_solution_files_rh)

        # Calculate the batch size for both hemispheres
        # Change 10 to your desired batch size
        batch_size_lh = total_files_lh // (total_files_lh // 10)

        # Change 10 to your desired batch size
        batch_size_rh = total_files_rh // (total_files_rh // 10)

        # Ensure batch size is a multiple of 10 (or your desired batch size) for both hemispheres
        while total_files_lh % batch_size_lh != 0:
            batch_size_lh -= 1

        while total_files_rh % batch_size_rh != 0:
            batch_size_rh -= 1

        # Initialize lists to store source estimates for both hemispheres
        stcs_lh = []
        stcs_rh = []

        # Load data in batches for both hemispheres
        for i in range(0, total_files_lh, batch_size_lh):
            batch_files_lh = inverse_solution_files_lh[i:i + batch_size_lh]
            batch_files_rh = inverse_solution_files_rh[i:i + batch_size_rh]

            for file_path_lh, file_path_rh in zip(batch_files_lh, batch_files_rh):
                try:
                    print(file_path_lh, file_path_rh)
                    stc_epoch_lh = mne.read_source_estimate(file_path_lh)
                    stc_epoch_rh = mne.read_source_estimate(file_path_rh)
                    stcs_lh.append(stc_epoch_lh)
                    stcs_rh.append(stc_epoch_rh)
                except Exception as e:
                    print(f"Error loading files {file_path_lh} or {file_path_rh}: {e}")

        # Load labels from the atlas
        labels = mne.read_labels_from_annot('fsaverage', parc='Schaefer2018_100Parcels_7Networks_order',
                                            subjects_dir=r'../data/in/')

        # Extract label time courses for both hemispheres
        label_time_courses = []  # Initialize a list to store label time courses
        
        print(stcs_lh, stcs_rh)
        for idx, (stc_lh, stc_rh) in enumerate(zip(stcs_lh, stcs_rh)):
            try:
                label_tc_lh = stc_lh.extract_label_time_course(
                    labels, src=src, mode='mean_flip')
                label_tc_rh = stc_rh.extract_label_time_course(
                    labels, src=src, mode='mean_flip')
                label_time_courses.extend([label_tc_lh, label_tc_rh])
                print(src)
            except Exception as e:
                print(f"Error extracting label time courses for iteration {idx}: {e}")
        else:  # This block will execute if the for loop completes without encountering a break statement
            print("All time courses have been successfully extracted!")

        # Convert label_time_courses to a NumPy array
        label_time_courses_np = np.array(label_time_courses)

        # If you prefer to save as a .csv file
        # Convert to DataFrame and save as .csv
        # label_time_courses_df = pd.DataFrame(label_time_courses_np)
        # label_time_courses_df.to_csv(os.path.join(output_dir, f"{subj}_label_time_courses.csv"), index=False)

        # Save the label time courses as a .npy file
        # Replace with your desired output directory
        label_time_courses_file = output_path + f"{subject}_label_time_courses.npy"
        
        np.save(label_time_courses_file, label_time_courses_np)

        ########################################################################################################################

#         # Plotting Time Courses
#         random_idx = np.random.randint(len(label_time_courses))
#         random_time_course = label_time_courses[random_idx]

#         plt.figure(figsize=(10, 6))
#         plt.plot(random_time_course)
#         plt.title(f'Time Course for Randomly Selected Region: {random_idx}')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
        

#         plt.savefig(output_path+'time_courses.png')
        
        
        
#         print('Connectivity Viz')
#         # Connectivity Visualization for left hemisphere
#         num_regions = len(label_time_courses[0])
#         connectivity_matrix = np.zeros((num_regions, num_regions))

#         for i in range(num_regions):
#             for j in range(num_regions):
#                 connectivity_matrix[i, j], _ = pearsonr(
#                     label_time_courses[0][i], label_time_courses[0][j])

#         plt.figure(figsize=(10, 10))
#         plt.imshow(connectivity_matrix, cmap='viridis', origin='lower')
#         plt.title('Connectivity Matrix')
#         plt.xlabel('Region')
#         plt.ylabel('Region')
#         plt.colorbar(label='Pearson Correlation')
        
#         print('Saving Connectivity Matrix')
#         plt.savefig(output_path+'connectivity_matrix.png')

#         # Average connectivity matrix across all epochs and hemispheres
#         # Initialize connectivity matrix
#         num_epochs_hemispheres = len(label_time_courses)
#         num_regions = label_time_courses[0].shape[0]
#         all_connectivity_matrices = np.zeros(
#             (num_epochs_hemispheres, num_regions, num_regions))

#         # Compute connectivity for each epoch and hemisphere
#         print('Computing connectivity')
#         print(num_epochs_hemispheres*num_regions*num_regions)
#         for k in range(num_epochs_hemispheres):
#             for i in range(num_regions):
#                 for j in range(num_regions):
#                     all_connectivity_matrices[k, i, j], _ = pearsonr(
#                         label_time_courses[k][i], label_time_courses[k][j])
                    
#                     print(k,i,j)

#         # Average across all epochs and hemispheres
#         avg_connectivity_matrix = np.mean(all_connectivity_matrices, axis=0)

#         # Visualization
#         plt.figure(figsize=(10, 10))
#         plt.imshow(avg_connectivity_matrix, cmap='viridis', origin='lower')
#         plt.title('Average Connectivity Matrix')
#         plt.xlabel('Region')
#         plt.ylabel('Region')
#         plt.colorbar(label='Pearson Correlation')
        
#         print('saving avg pearson Connectivity Matrix')
#         plt.savefig(output_path+'connectivity_avg_matrix.png')

        ########################################################################################################################
        # All-to-all connectivity analysis


        # Load the label time courses
        label_time_courses_file = output_path+ f"{subject}_label_time_courses.npy"

        label_time_courses = np.load(label_time_courses_file)

        # Load labels from the atlas
        labels = mne.read_labels_from_annot('fsaverage', parc='Schaefer2018_100Parcels_7Networks_order',
                                            subjects_dir=r'../data/in/')

        # Group labels by network
        networks = {}
        for label in labels:
            # Extract network name from label name (assuming format: 'NetworkName_RegionName')
            network_name = label.name.split('_')[0]
            if network_name not in networks:
                networks[network_name] = []
            networks[network_name].append(label)

        # Organize regions by their network affiliations and extract the desired naming convention
        ordered_regions = []
        network_labels = []  # This will store the network each region belongs to

        for label in labels:
            # Extract the desired naming convention "PFCl_1-lh" from the full label name
            parts = label.name.split('_')
            region_name = '_'.join(parts[2:])
            ordered_regions.append(region_name)

            # Extract the network name and store it in network_labels
            network_name = parts[2]
            network_labels.append(network_name)

        # Compute cross-correlation between all pairs of regions across windows
        print('Computing cross corelation')
        # Time-resolved dPLI computation
        sampling_rate = 512  # in Hz
        window_length_seconds = 1
        step_size_seconds = 0.5
        
        # Total duration in samples
        num_epochs_per_hemisphere = label_time_courses.shape[0] / 2  # Assuming the structure is the same as label_time_courses in Code 2
        duration_per_epoch = label_time_courses.shape[2] / sampling_rate
        total_duration_samples = int(num_epochs_per_hemisphere * duration_per_epoch * sampling_rate)
        
        window_length_samples = int(window_length_seconds * sampling_rate)
        step_size_samples = int(step_size_seconds * sampling_rate)
        
        num_windows = int((total_duration_samples - window_length_samples) / step_size_samples) + 1
        windowed_dpli_matrices = []
        windowed_cross_correlation_matrices = []
        
        for win_idx in range(num_windows):
            print('Processing', win_idx)
            start_sample = win_idx * step_size_samples
            end_sample = start_sample + window_length_samples
        
            # Check if end_sample exceeds the total number of samples
            if end_sample > total_duration_samples:
                break
        
            # Calculate epoch and sample indices
            start_epoch = start_sample // label_time_courses.shape[2]
            start_sample_in_epoch = start_sample % label_time_courses.shape[2]
            end_epoch = end_sample // label_time_courses.shape[2]
            end_sample_in_epoch = end_sample % label_time_courses.shape[2]
        
            # Extract data across epochs
            if start_epoch == end_epoch:
                windowed_data = label_time_courses[start_epoch, :, start_sample_in_epoch:end_sample_in_epoch]
            else:
                first_part = label_time_courses[start_epoch, :, start_sample_in_epoch:]
                samples_needed_from_second_epoch = window_length_samples - first_part.shape[1]
                second_part = label_time_courses[end_epoch, :, :samples_needed_from_second_epoch]
                windowed_data = np.concatenate((first_part, second_part), axis=1)
        
            # dPLI computation
            dpli_result = compute_dPLI(windowed_data)
            windowed_dpli_matrices.append(dpli_result)
        
        # Check the number of windows in the list
        num_of_windows = len(windowed_dpli_matrices)
        print(f"Total number of windows: {num_of_windows}")
        
        # Calculating the average connectivity across all windows (or choose a specific window)
        #chosen_window = 0 # Choose a window
        dPLI_matrix = windowed_dpli_matrices # can do windowed_dpli_matrices[chosen_window] to only choose one window
        G_dPLI_list = [nx.convert_matrix.from_numpy_array(matrix) for matrix in windowed_dpli_matrices]
        
        
        print('Working on alphas')
        # Determining the optimal alpha value for disparity filter before applying thresholding
        
        # Disparity filter

        # Determining alpha
        alphas = np.linspace(0.001, 0.1, 100)  # Example range of alphas to test
        avg_connectivities = []
        
        for alpha in alphas:
            avg_conn_for_alpha = []
            for G_dPLI in G_dPLI_list:  # Use the already converted graphs
                
                print('Finding optimal alpha')
                G_dPLI_thresholded = disparity_filter(G_dPLI, alpha=alpha)
        
                # Convert the thresholded graph back to a matrix
                dPLI_matrix_thresholded = nx.convert_matrix.to_numpy_array(G_dPLI_thresholded)
        
                # Compute the average functional connectivity of the thresholded matrix
                avg_conn = np.mean(dPLI_matrix_thresholded)
                avg_conn_for_alpha.append(avg_conn)
        
            # Compute the average of averages for this alpha
            avg_connectivities.append(np.mean(avg_conn_for_alpha)) # AKA, for this alpha, what's the typical (or average) connectivity value across all windows?
        
        # Find the alpha that gives the most stable average connectivity
        optimal_alpha = alphas[np.argmin(np.diff(avg_connectivities))]
        
        print(f"Optimal alpha: {optimal_alpha}")
        
        # Thresholding the connectivity matrix
        G_dPLI_thresholded_list = [threshold_graph_by_density(G) for G in G_dPLI_list]  # Use the optimal alpha value here
        
        ########################################################################################################################
        # Next --> fc between groups and directed minimum spanning tree to examine nature of the fc difference between groups

0 files missing from root.txt in /home/prakhar7/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage
Using matplotlib as 2D backend.
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
101 EC
../data/out/subjects/101/EC/stc/101_inversesolution_epoch14.fif-lh.stc ../data/out/subjects/101/EC/stc/101_inversesolution_epoch33.fif-rh.stc
../data/out/subjects/101/EC/stc/101_inversesolution_epoch33.fif-lh.stc ../data/out/subjects/101/EC/stc/101_inversesolution_epoch38.fif-rh.stc
../data/out/subjects/101/EC/stc/101_inversesolution_epoch50.fif-lh.stc ../data/out/subjects/101/EC/stc/101_inversesolution_epoch18.fif-rh.stc
../data/out/subjects/101/EC/stc/101_inversesolution_epoch21.fif-lh.stc ../data/out/subjects/101/EC/stc/101_inversesolution_epoch31.fif-rh.stc
../data/out/subjects/101/EC/stc/101_inversesolution_epoch42.fif-lh.stc ../data/out/subjects/101/EC/stc/101_inversesolution_ep

   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<SourceEstimate | 20484 vertices, tmin : -500.0 (ms), tmax : 500.0 (ms), tstep : 1.953125 (ms), data shape : (20484, 513), ~40.2 MB>, <SourceEstimate | 20484 vertices, tmin : -500.0 (ms), tmax : 500.0 (ms), tstep : 1.953125 (ms), data shape : (20484, 513), ~40.2 MB>, <SourceEstimate | 20484 vertices, tmin : -500.0 (ms), tmax : 500.0 (ms), tstep : 1.953125 (ms), data shape : (20484, 513), ~40.2 MB>, <SourceEstimate | 20484 vertices, tmin : -500.0 (ms), tmax : 500.0 (ms), tstep : 1.953125 (ms), data shape : (20484, 513), ~40.2 MB>, <SourceEstimate | 20484 vertices, tmin : -500.0 (ms), tmax : 500.0 (ms), tstep : 1.953125 (ms), data shape : (20484, 513), ~40.2 MB>, <SourceEstimate | 20484 vertices, tmin : -5

Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertic

Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertices=163842, n_used=10242>] MRI (surface RAS) coords, subject 'fsaverage', ~21.9 MB>
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
<SourceSpaces: [<surface (lh), n_vertices=163842, n_used=10242>, <surface (rh), n_vertic

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
Computing cross corelation
Processing 0
Computing dPLI
[[1.83680585e-14 2.01720405e-14 2.14687616e-14 ... 2.04087574e-14
  2.19071231e-14 2.21807927e-14]
 [7.04400985e-15 7.62371581e-15 8.32286019e-15 ... 9.01718750e-15
  1.01864648e-14 1.08103766e-14]
 [2.45988244e-14 2.58460618e-14 2.68820949e-14 ... 2.63200496e-14
  2.65423517e-14 2.63217674e-14]
 ...
 [3.09118982e-14 3.63545728e-14 4.17833933e-14 ... 3.75112945e-14
  4.43660577e-14 4.88094130e-14]
 [2.58782203e-14 2.89943579e-14 3.14168247e-14 ... 2.94401242e-14
  3.10342775e-14 3.14452376e-14]
 [2.24553720e-14 2.49703178e-14 2.70731551e-14 ... 2.60019396e-14
  2.80232888e-14 2.85783936e-14]]
Processing 1
Computing 

...

Processing 56
Computing dPLI
[[3.2331393e-14 3.3723146e-14 3.4756327e-14 ... 1.7651425e-14
  1.8384481e-14 2.0435002e-14]
 [1.1428549e-14 1.1962185e-14 1.2433992e-14 ... 8.5516412e-15
  7.4011071e-15 7.1858184e-15]
 [2.9633197e-14 3.1465106e-14 3.3071504e-14 ... 2.6376262e-14
  2.5356384e-14 2.5084515e-14]
 ...
 [4.3025130e-14 4.6640084e-14 5.0821093e-14 ... 1.6333538e-14
  1.4900231e-14 1.5428822e-14]
 [3.7408482e-14 3.8506646e-14 3.9382970e-14 ... 2.3324694e-14
  2.3181986e-14 2.4134073e-14]
 [3.3031148e-14 3.4315165e-14 3.4990365e-14 ... 2.0000972e-14
  1.9690607e-14 2.1017332e-14]]
Processing 57
Computing dPLI
[[2.1076302e-14 2.0910399e-14 2.0022856e-14 ... 2.1595188e-14
  2.1495228e-14 2.2634071e-14]
 [8.3408969e-15 9.7432912e-15 1.1027799e-14 ... 1.0420986e-14
  1.0544225e-14 1.0823075e-14]
 [1.9219172e-14 2.0430772e-14 2.2168318e-14 ... 2.9610639e-14
  3.1852996e-14 3.2603897e-14]
 ...
 [3.1199093e-14 3.1990181e-14 3.3426858e-14 ... 6.0532972e-14
  5.9454564e-14 5.4284509e-14]
 

Processing 70
Computing dPLI
[[2.3444849e-14 2.3686099e-14 2.4421501e-14 ... 3.7317914e-14
  3.6040686e-14 3.5041787e-14]
 [2.0585355e-14 2.0936754e-14 2.0696068e-14 ... 1.4550922e-14
  1.3607262e-14 1.2963593e-14]
 [3.5387373e-14 3.7399408e-14 3.8261407e-14 ... 3.8341566e-14
  3.7792726e-14 3.7377741e-14]
 ...
 [4.0534687e-14 4.0518051e-14 4.1332843e-14 ... 6.6107981e-14
  6.3812345e-14 6.0193963e-14]
 [3.5388223e-14 3.5651827e-14 3.6118295e-14 ... 4.8784273e-14
  4.6898585e-14 4.4782097e-14]
 [3.2498869e-14 3.1249576e-14 3.0468930e-14 ... 4.1777251e-14
  4.0296983e-14 3.8672790e-14]]
Processing 71
Computing dPLI
[[2.5149477e-14 2.6347404e-14 2.7210634e-14 ... 3.0271311e-14
  2.7451287e-14 2.3965898e-14]
 [1.1341480e-14 1.0990902e-14 1.0915277e-14 ... 1.4870836e-14
  1.4135357e-14 1.3306140e-14]
 [2.8572021e-14 2.8529642e-14 2.8115390e-14 ... 4.5796286e-14
  4.4006424e-14 4.1934521e-14]
 ...
 [4.8052742e-14 4.9266045e-14 5.0620722e-14 ... 5.5711878e-14
  5.0857461e-14 4.7371283e-14]
 

Processing 86
Computing dPLI
[[2.26420665e-14 2.46310744e-14 2.58521164e-14 ... 1.19314484e-14
  1.33258993e-14 1.38887425e-14]
 [9.72518253e-15 8.52687316e-15 8.25938185e-15 ... 3.55074623e-15
  3.06749742e-15 2.76066545e-15]
 [2.06629063e-14 2.25944988e-14 2.48874577e-14 ... 2.28556001e-14
  2.24473336e-14 2.23394504e-14]
 ...
 [2.55645335e-14 2.58810070e-14 2.71834337e-14 ... 2.53725992e-14
  2.45421732e-14 2.50624039e-14]
 [2.38203419e-14 2.58296192e-14 2.78714328e-14 ... 1.68948345e-14
  1.86770324e-14 2.01068969e-14]
 [2.08631601e-14 2.31524106e-14 2.51005847e-14 ... 1.49383933e-14
  1.57202810e-14 1.65055568e-14]]
Processing 87
Computing dPLI
[[1.6560919e-14 1.9229701e-14 2.4511902e-14 ... 1.6264266e-14
  1.6796917e-14 1.8455687e-14]
 [7.9475060e-15 8.9195585e-15 1.0538827e-14 ... 8.7331308e-15
  8.6690163e-15 8.9903425e-15]
 [3.2721777e-14 3.5134435e-14 3.8183588e-14 ... 2.8231291e-14
  2.8491857e-14 2.8356212e-14]
 ...
 [2.5573670e-14 2.7619700e-14 3.2533353e-14 ... 3.8575263e

Processing 100
Computing dPLI
[[3.7779635e-14 3.4559229e-14 3.1037425e-14 ... 1.6065984e-14
  1.5453698e-14 1.5053986e-14]
 [1.5008714e-14 1.4735285e-14 1.3870419e-14 ... 6.6004437e-15
  6.2718940e-15 5.9317717e-15]
 [3.0952498e-14 3.2095329e-14 3.3123257e-14 ... 2.2598722e-14
  1.9110752e-14 1.5993087e-14]
 ...
 [6.9945602e-14 6.5523833e-14 5.9867943e-14 ... 1.5855479e-14
  1.2938684e-14 1.1472557e-14]
 [4.5214832e-14 4.2279013e-14 3.9146959e-14 ... 2.2425517e-14
  2.1312924e-14 2.0369077e-14]
 [4.0445322e-14 3.7633230e-14 3.4583220e-14 ... 1.9711529e-14
  1.8671339e-14 1.7864514e-14]]
Processing 101
Computing dPLI
[[3.4547201e-14 3.3199311e-14 3.2008440e-14 ... 2.4818672e-14
  2.2636136e-14 2.0654915e-14]
 [1.4265080e-14 1.3393728e-14 1.2685253e-14 ... 1.2266316e-14
  1.1330794e-14 1.0554988e-14]
 [3.4080506e-14 3.4690079e-14 3.5735636e-14 ... 3.9265961e-14
  3.5714646e-14 3.2464588e-14]
 ...
 [5.8176147e-14 5.6284621e-14 5.4597514e-14 ... 4.3144023e-14
  3.9356197e-14 3.8420449e-14]

Processing 114
Computing dPLI
[[1.96431548e-14 1.98559719e-14 2.00866325e-14 ... 3.93808419e-14
  3.87688267e-14 3.74442739e-14]
 [1.19651425e-14 1.27503311e-14 1.27763062e-14 ... 1.14826912e-14
  1.07075722e-14 9.76055293e-15]
 [3.19437232e-14 3.05222800e-14 2.99391622e-14 ... 3.15785741e-14
  3.10808440e-14 3.00539589e-14]
 ...
 [3.51587622e-14 3.46310437e-14 3.44393906e-14 ... 5.69654255e-14
  5.67114003e-14 5.49142132e-14]
 [3.26891834e-14 3.26584496e-14 3.28520407e-14 ... 4.21925542e-14
  4.23970212e-14 4.17727241e-14]
 [2.73157182e-14 2.72559295e-14 2.76033384e-14 ... 3.86636557e-14
  3.82738817e-14 3.70785454e-14]]
Total number of windows: 115
